**Project 3: Wine Prediction Redux**
---

Using the provided characteristics, use clustering algorithms to pre-process the data to garner a better prediction using the selected model.

**Imports & Declaring Constants**

In [ ]:
### Tool imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix

### sklearn imports
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.model_selection import GridSearchCV

### Local imports
import utils.data_handling_lib as dhl
import utils.graphics_lib as gl
import utils.clustering_tools_lib as ctl

from utils.data_handling_lib import RANDOM_STATE

**Load an Explore Copy**

In [ ]:
explore_raw_whole = dhl.load_data()

explore_raw = explore_raw_whole[3000:]

explore_raw.head()

**Split dataset**

In [ ]:
X_train_raw, y_train_raw, X_test_raw, y_test_raw = dhl.strat_split_dataset(X=explore_raw_whole, label_id="quality")

**Scatter plots**

*Running this scatter matrix borks the rest of the graphs, refer to the exported image and heatmap*

In [ ]:
#attributes = ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides",
#              "free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol",
#              "quality","color"]
#scatter_matrix(explore_raw[attributes], figsize=(12, 8))

**Correlation matrix**

In [ ]:
corr_matrix = explore_raw.corr()
corr_matrix["quality"].sort_values(ascending=False)

In [ ]:
gl.plot_corr_matrix_heatmap(corr_matrix=corr_matrix, dataset=explore_raw, savefig=True)

First look:
    Most significant correlations for *quality* are *alcohol, density, chlorides, volatile acidity*

**KMeans test**

In [ ]:
param_grid = dict(kmeans__n_clusters=[_ for _ in range(2,15)])

kmeans = KMeans(n_clusters=8, random_state=RANDOM_STATE)

In [ ]:
drop_features_1 = ["free sulfur dioxide", "total sulfur dioxide", "pH", "citric acid", "residual sugar", "sulphates", "volatile acidity"]
X_train_drop1, X_test_drop1 = X_train_raw.drop(drop_features_1, axis=1), X_test_raw.drop(drop_features_1, axis=1)

drop_1_sil_scores = ctl.get_silhouette_scores(X_train_drop1, range_upper=10)

gl.plot_silhouette_scores(silhouette_scores=drop_1_sil_scores, var_name="X_train_drop_1")

In [ ]:
drop_features_2 = ["fixed acidity", "volatile acidity", "density", "residual sugar", "chlorides", "citric acid", "color"]
X_train_drop2, X_test_drop2 = X_train_raw.drop(drop_features_2, axis=1), X_test_raw.drop(drop_features_2, axis=1)

drop_2_sil_scores = ctl.get_silhouette_scores(X_train_drop2, range_upper=10)

gl.plot_silhouette_scores(silhouette_scores=drop_2_sil_scores, var_name="X_train_drop_2")

Second highest performing set.

In [ ]:
drop_features_3 = ["fixed acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","color"]
X_train_drop3, X_test_drop3 = X_train_raw.drop(drop_features_3, axis=1), X_test_raw.drop(drop_features_3, axis=1)

drop_3_sil_scores = ctl.get_silhouette_scores(X_train_drop3, range_upper=10)

gl.plot_silhouette_scores(silhouette_scores=drop_3_sil_scores, var_name="X_train_drop_3")

Ideal number of clusters is 2, seems to be highest performing set.

*change view: including attribs rather than dropping*

In [ ]:
include_features_4 = ["total sulfur dioxide", "free sulfur dioxide", ]